<a href="https://colab.research.google.com/github/fcoliveira-utfpr/agrometeorologia/blob/main/05_bh_climatologico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inserindo informações**
---

In [2]:
# ============================================================
municipio = "Dourados" ###### Nome aqui entre aspas

estado = "MS" ###### Sigla do estado entre aspas

# ============================================================
# ============================================================

#importando bibliotecas
import numpy as np
import pandas as pd

# **Carrega base climática TerraClimate 1991-2020**
---

In [11]:
# Carrega base climática local
# ============================================================
url = "https://raw.githubusercontent.com/fcoliveira-utfpr/agrometeorologia/refs/heads/main/TerraClimate_PR_PET_Normal_1991_2020.csv"
df = pd.read_csv(url)

# Separa colunas categóricas
cidade = df['Município']
uf = df['Estado']

# Converte valores com vírgula para ponto e numérico
df_valores = df.drop(columns=['Município', 'Estado'])
df_valores = df_valores.replace({',': '.'}, regex=True)
df_valores = df_valores.apply(pd.to_numeric, errors='coerce')

# Reconstrói o df
df = df_valores.copy()
df['Município'] = cidade
df['Estado'] = uf

# Filtra município e estado
df = df[(df['Município'] == municipio) & (df['Estado'] == estado)].copy()

# Correção: Alterar a lista de meses para corresponder ao formato das colunas (ex: 'PET_01')
meses = [f"{i:02d}" for i in range(1, 13)]

pet = [df[f"PET_{m}"].values[0] for m in meses]
chuva = [df[f"PR_{m}"].values[0] for m in meses]

mes = ["jan","fev","mar","abr","maio","jun","jul","ago","set","out","nov","dez"]

df_bh = pd.DataFrame(
    {
      'Meses': mes,
      'P (mm/mês)': chuva,
      'ETP (mm/mês)': pet,
    }
)
df_bh

,Meses,P (mm/mês),ETP (mm/mês)
0,jan,158.933333,133.543333
1,fev,152.400000,119.686667
2,mar,111.533333,123.853333
3,abr,111.600000,104.133333
4,maio,120.000000,85.313333
5,jun,75.333333,76.930000
6,jul,50.233333,88.060000
7,ago,46.233333,115.263333
8,set,101.400000,124.133333
9,out,170.566667,130.810000
